In [1]:
import requests
import re
import bs4
import pandas as pd

In [2]:
# check4nextpage returns True or False

def check4nextpage(URL):
    
    res = requests.get(URL)
    soup = bs4.BeautifulSoup(res.text,'html.parser')
    nextButton = soup.find_all('input', type = 'button', value = 'Next 100', limit = 1)
    prevButton = soup.find_all('input', type = 'button', value = 'Previous 100', limit = 1)
    
    if nextButton:
        return True
    
    else:
        return False

In [3]:
# get_descriptions pulls all the 10-Ks and 10-Qs on a certain page

def get_descriptions(URL):
    someRes = requests.get(URL)
    someSoup = bs4.BeautifulSoup(someRes.text, 'html.parser')

    descriptions = []
    
    for i in someSoup.find_all('tr'):
        for j in i.find_all('a', attrs = {'href' : re.compile('Archives')}):
            if '/A' in i.text:
                break
            if '10-Q' in i.text or '10-K' in i.text:
                descriptions.append(i.text)
                
        # Clean Up
    filingType = []
    for i in descriptions:
        if '10-Q' in i:
            filingType.append('10-Q')
        if '10-K' in i:
            filingType.append('10-K')
    
    return filingType

In [4]:
# get_links pulls all the links to the 10-Ks and 10-Qs on a certain page

def get_links(URL):
    someRes = requests.get(URL)
    someSoup = bs4.BeautifulSoup(someRes.text,'html.parser')
    
    links = []
    for i in someSoup.find_all('tr'):
        for j in i.find_all('a', attrs = {'href' : re.compile('Archives')}):
            if '/A' in i.text:
                break
            if '10-Q' in i.text or '10-K' in i.text:
                links.append(j.get('href'))
    
    return links

In [5]:
def link_generator(tickersymbol):
    file = open('ticker.txt','r')
    lines = file.readlines()

    d = {}
    for i in lines: 
        (key, val) = i.split()
        d[(key)] = val
    
    CIK = d[str(tickersymbol.lower())]
    
    while len(CIK) <= 10:
        CIK = '0' + CIK
    
    index = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=#REPLACEME#&owner=exclude&count=100'
    URL = index.replace('#REPLACEME#',CIK)
    
    return URL

In [6]:
ticker = input('Type in a Ticker symbol ')

URL = link_generator(ticker)

URL_og = URL

Type in a Ticker symbolaapl


In [7]:
pages = [URL] # initiates a list pages with the original URL
n = 100 # initiates a variable that will be used to navigate the following pages

for i in pages: # uses check4nextpage to create a list of the all the links that exist for a certain name
    
    while check4nextpage(URL) == True:
        newURL = URL_og + '&start=' + str(n)
        URL = newURL
        n += 100
        pages.append(newURL)
        
    if check4nextpage(URL) == False:
        break

In [8]:
filings = []
for i in pages: # iterates through all the pages to pull the filings types
    filings.extend(get_descriptions(i))

In [9]:
links = []
for i in pages: # iterates through all the pages to pull the links the filing types
    links.extend(get_links(i))

In [10]:
df = pd.DataFrame({'Filings': filings, 'Links': links})
df

,Filings,Links
0,10-K,/Archives/edgar/data/320193/000032019320000096...
1,10-Q,/Archives/edgar/data/320193/000032019320000062...
2,10-Q,/Archives/edgar/data/320193/000032019320000052...
3,10-Q,/Archives/edgar/data/320193/000032019320000010...
4,10-K,/Archives/edgar/data/320193/000032019319000119...
...,...,...
104,10-Q,/Archives/edgar/data/320193/0000320193-95-0000...
105,10-Q,/Archives/edgar/data/320193/0000320193-95-0000...
106,10-K,/Archives/edgar/data/320193/0000320193-94-0000...
107,10-Q,/Archives/edgar/data/320193/0000320193-94-0000...


In [11]:
# The following is still a work in progress

root = 'https://www.sec.gov'

In [12]:
filingURL = []

for i in links: # create a list of all the filingURLs
    filingURL.append(root + i)

In [13]:
def getdirectlink(URL):
    
    someRes = requests.get(URL)
    someSoup = bs4.BeautifulSoup(someRes.text, 'html.parser')
    
    for i in someSoup.find_all('table', class_ = 'tableFile'):
        for j in i.find_all('a', attrs = {'href':re.compile('Archives')}, limit = 1):
            return (j.get('href'))

In [14]:
directlinks = []

for i in filingURL:
    directlinks.append(getdirectlink(i))

In [15]:
finalURLs = []

for i in directlinks[0:28]:
    someURL = root + i
    if '/ix?doc=/' in someURL:
        someURL = someURL.replace('/ix?doc=','')
    finalURLs.append(someURL)
    

In [21]:
filename = str(ticker.upper()) +'_URL_List'

f = open(filename,'x')

with open(filename,'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in finalURLs)